In [1]:
#Librerias que vamos a utilizar
import numpy as np
import pyhdf.SD
from pyhdf.SD import SD,SDC #Necesitamos la libreria pyhdf para leer archivos y SD una clase
import glob #una libreria que usamos
import pandas as pd
import matplotlib.pyplot as plt
import re #liberia para regex que luego usaremos en texto

In [2]:
#Lectura de archivos de calibrated radiance

#Ruta donde se encuentran los archivos hdf
DataPath = 'archivos'

#Generamos una variable con todos los archivos de la carpeta
Filelist = glob.glob(DataPath+'/*')

#Mostramos por pantalla
print(Filelist)

['archivos/MOD021KM.A2022224.1430.006.2022225012914.hdf', 'archivos/MOD021KM.A2022091.1415.006.2022092022447(2).hdf']


In [6]:
#Elegimos el archivo con el que vamos a trabajar segun el indice 

n_file = 1

#Abrimos el archivo: SD una clase de la libreria pyhdf junto con el READ lo utilizamos
#para leer el archivo

File = SD(Filelist[n_file],SDC.READ)

def lee_archivo(File):
    archivo = SD(File,SDC.READ)
    return archivo
    

#Mostramos informacion de nuestro archivo hdf

print(File.info()) 

Nuestro archivo hdf contiene 31 sds (Datasets). Para ver esto de forma mas detallada utilizamos la funcion "visualiza_hdf" que nos devuelve una tabla con la informacion de los sds.

#Metadata 

Vamos a trabajar con lo que se conoce como el metadata de nuestro archivo HDF. Esto lo hacemos debido a que dentro de él, tenemos algunos datos interesantes como lo son la fecha y la hora en la que se scaneo la información. Lo vamos a obtener utilizando el metodo attributes() que luego volveremos a ver. 

In [4]:
#File.attributes() #ejecutar para observar el metadata

Se trata de un diccionario que contiene informacion. Generemos una tabla donde podamos observarla un poco mejor

In [5]:
#pd.DataFrame(File.attributes().keys()) #ejecutar para ver en formato tabla

De este diccionario nos va a interesar el contenido de "CoreMetadata.0" que segun la documentación de https://ladsweb.modaps.eosdis.nasa.gov/filespec/MODIS/5/MOD03 tiene información sobre hora y fecha.
Al extraer el metadata lo obtendremos en formato "string".

In [6]:
meta_data = File.attributes()["CoreMetadata.0"]
#print(type(meta_data))
#print(meta_data)

In [7]:
#Formato de la fecha segun la documentacion YYYY-MM-DD
patron_fecha = r'(\d{4})-(\d{2})-(\d{2})'

#Busco en el texto la primera fecha. Conociendo la manera en la que se almacena la 
#informacion. Se que va a encontrar la fecha correcta. Solo almacene 5 min de datos
fecha = re.search(patron_fecha,meta_data)[0]

print(fecha)

2022-04-02


In [8]:
#Una funcion que nos servira para hallar fecha siguiendo los anteriores pasos

def hora_archivo(File):
    metadata = File.attributes()["CoreMetadata.0"]
    patron_horario = r'(\d{4})-(\d{2})-(\d{2})'
    fecha = re.search(patron_fecha,meta_data)[0]
    return fecha

In [9]:
def visualiza_hdf(File):
    
    #El modulo datasets (proviene de la libreria pyhdf) me devuelve informacion sobre que
    #hay dentro del hdf. 
    datasets = {"Datasets": list(File.datasets().keys())}
    
    #Lo transformamos a un dataframe para visualizar
    info_file = pd.DataFrame(datasets)
    
    return info_file

visualiza_hdf(File)

,Datasets
0,Latitude
1,Longitude
2,EV_1KM_RefSB
3,EV_1KM_RefSB_Uncert_Indexes
4,EV_1KM_Emissive
5,EV_1KM_Emissive_Uncert_Indexes
6,EV_250_Aggr1km_RefSB
7,EV_250_Aggr1km_RefSB_Uncert_Indexes
8,EV_250_Aggr1km_RefSB_Samples_Used
9,EV_500_Aggr1km_RefSB


Para calcular nuestro indice vamos a necesitar el sds de "EV_500_Aggr1km_RefSB" en el cual vamos a tener valores de reflectancia. Y mas aun las bandas que necesitamos.

In [10]:
#Funcion que pide como argumento un archivo hdf y el dataset que queramos

def selecciona_sds_hdf(File,sds):
    
    #Seleccionamos el dataset con "select" metodo de la libreria pyhdf
    sds_seleccionado = File.select(sds)
    
    return sds_seleccionado

In [11]:
#Toma como argumento un archivo hdf y el dataset que desee extraer

def extrae_datos_hdf(File,sds):
    
    #Seleccionamos el dataset con la funcion
    dataset = selecciona_sds_hdf(File,sds)
    
    #Extraemos los datos con el metodo get de pyhdf
    sds_datos = dataset.get()
    
    #Devuelve los datos y su dimension
    return sds_datos,sds_datos.shape

Para ver que informacion "atributos" tenemos en nuestro dataset vamos a utilizar la funcion "muestra_atributos_sds" la cual nos entrega una tabla donde los visualizamos.

In [12]:
def selecciona_atributos_sds(File,sds):
    
    #Seleccionamos el dataset
    dataset = selecciona_sds_hdf(File,sds)
    
    #Utilizamos el metodo attribute, nos devuelve un diccionario con una descripcion
    #global de cada atributo adjunto al archivo hdf
    dic_atributos = dataset.attributes()
    
    return dic_atributos

In [13]:
#Input : File y un dataset
def muestra_atributos_sds(File,sds):
    
    dic_atributos = selecciona_atributos_sds(File,sds)
        
    #Tomamos las llaves y valores del diccionario y los transformamos en listas
    #para luego generar una tabla y visualizarlo
    columna1 = list(dic_atributos.keys())
    columna2 = list(dic_atributos.values())
    
    #Generamos una tabla/dataframe
    df = pd.DataFrame({"":columna1,"Atributos":columna2})
    df = df.set_index("")
    
    #Regresa una tabla donde se muestran los atributos del dataset
    return df

In [ ]:
#muestra_atributos_sds(File,"EV_500_Aggr1km_RefSB")

In [ ]:
#Se elige un file hdf, sds: el dataset que queramos y el atributo del dataset
def extrae_atributos_sds(File,sds,atributo):
    
    #Utilizamos la funcion para generar un diccionario con los atributos
    dic_atributos = selecciona_atributos_sds(File,sds)
    
    #Del diccionario elegimos el atributo que queremos
    attr = dic_atributos[atributo]
    
    #Devuelve una lista con los valores de mi atributo
    return attr

In [14]:
#Extrae los valores de la banda seleccionada ya convertidos a reflectancia

#File hdf, sds: dataset,canal deseado, nombre de atributo con las bandas
def extrae_banda(File,sds,band,nombre_atributo):
    
    #Extraemos los valores de reflectancia sin conversion
    REF = extrae_datos_hdf(File,sds)[0]
    
    #Extraemos los numeros de bandas que tenemos. Ej:band 1, band2
    #Spliteamos para operar en ellas
    n_bands = extrae_atributos_sds(File,sds,nombre_atributo).split(",")
    
    #Elegimos la banda
    
    posicion_band = n_bands.index(str(band))
    
    
    #Hacemos la conversion
    
    rad_off_ref= extrae_atributos_sds(File,sds,"reflectance_offsets")[posicion_band]
    
    rad_scales_ref = extrae_atributos_sds(File,sds,"reflectance_scales")[posicion_band]
    
    canal = rad_scales_ref * (REF[posicion_band,:,:] - rad_off_ref)
    
    return canal

In [15]:
def indice_nbr(File,sds,nombre_atributo):
    
    #Extraemos las bandas con las cuales vamos a trabajar
    canal_5 = extrae_banda(File,sds,5,nombre_atributo)
    canal_7 = extrae_banda(File,sds,7,nombre_atributo)
    
    #Calculamos el indice NBR
    nbr = (canal_5 - canal_7) / (canal_5 + canal_7)
    
    #array con los valores de nbr
    return nbr  


#Geolocalizacion

In [16]:
#Ruta donde se encuentran los archivos hdf
DataPath = 'geolocation'

#Generamos una variable con todos los archivos de la carpeta
Filelist_geo = glob.glob(DataPath+'/*')

#Mostramos por pantalla
print(Filelist_geo)

['geolocation/MOD03.A2022091.1415.006.2022091212727(1).hdf', 'geolocation/MOD03.A2022091.1415.006.2022091212727.hdf']


In [17]:
#Elegimos el archivo con el que vamos a trabajar segun el indice 

n_file = 0

#Abrimos el archivo

File_geo = SD(Filelist_geo[n_file],SDC.READ)

#Mostramos informacion de nuestro archivo hdf

print(File_geo.info()) 

(46, 27)


In [18]:
#visualiza_hdf(File_geo)